In [174]:
## Imports
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
from datetime import datetime
import os
import urllib.request
import nltk
nltk.download("wordnet")

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', "organization", "nashville", "football", "baseball", "receive"])

try:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
except Exception as e:
    os.system("python -m spacy download en")
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aantonison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.

In [175]:
def download_project_maslow_files():
    """The purpose of this function is to handle getting the nonprofit.txt and nonprofit_text.txt
    files.  This functin will create a data directory, add a .gitignore file so
    source control does not pick up the txt files, and then check to see
    if either nonprofit.txt and nonprofit_text.txt have been downloaded yet.
    If not, it will proceed to download the files.
    """

    if os.path.isdir("data") is False:
        print("data/ folder does not exist, creating it now")
        os.mkdir("data")

    if os.path.isfile("data/.gitignore") is False:
        os.system("echo *.txt > data/.gitignore")

    if os.path.isfile("data/nonprofit.txt") is False:
        print("Downloading nonprofit.txt...")
        urllib.request.urlretrieve("https://mtsu-dsi-hackathon-2022.s3.amazonaws.com/nonprofit.txt", 'data/nonprofit.txt')

    if os.path.isfile("data/nonprofit_text.txt") is False:
        print("Downloading nonprofit_text.txt...")
        urllib.request.urlretrieve("https://mtsu-dsi-hackathon-2022.s3.amazonaws.com/nonprofit_text.txt", 'data/nonprofit_text.txt')

In [176]:
def get_non_profit_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit
    """

    download_project_maslow_files()

    col_types = {
        "nonprofit_id": "Int64",
        "reporting_year": "Int64",
        "ein": "Int64",
        "businessname": "str",
        "phone": "str",
        "address1": "str",
        "address2": "str",
        "city": "str",
        "stabbrv": "str",
        "zip": "str"
      }

    df = pd.read_csv("data/nonprofit.txt", sep = "|", dtype=col_types)

    return df
df_np = get_non_profit_df()
df_np = df_np[["nonprofit_id", "stabbrv"]]


In [177]:
def get_non_profit_text_df():
    """This function will first run the download_project_maslow_files() to check to
    see if the files have been downloaded to the `data/` folder.  If not,
    it will download the files.
    Then, it will read in the file and return a dataframe with the nonprofit_text
    """

    download_project_maslow_files()

    col_types = {
        "nonprofit_text_id": "Int64",
        "reporting_year": "Int64",
        "nonprofit_id": "Int64",
        "grouptype": "str",
        "description": "str"
    }

    df = pd.read_csv("data/nonprofit_text.txt", sep = "|", encoding="cp1252", dtype=col_types)

    return df

np_text_df = get_non_profit_text_df()

In [178]:
# merge onto the nonprofit dataset and filter down to just TN
merged_df = np_text_df.merge(df_np, on="nonprofit_id")
filtered_df = merged_df[merged_df["stabbrv"] == "TN"].copy(deep=True)
filtered_df = filtered_df[["nonprofit_id", "description"]]

In [179]:
# remove instances where the description is duplicated across
# multiple categories
de_dup_df = filtered_df.drop_duplicates().copy(deep=True)
de_dup_df["description_word_length"] = de_dup_df["description"].str.split().str.len()
np_desc_populated = de_dup_df[de_dup_df["description_word_length"] > 10].copy(deep=True)
np_desc_populated.head()

,nonprofit_id,description,description_word_length
82,336076,Donation to Boy Scout Troop 441. The Troop has...,14.0
2374,5896,THE FOUNDATION'S PRIMARY OBJECTIVE IS TO SAVE ...,80.0
2855,369999,THE AUTISM FOUNDATION OF TENNESSEE'S PRIMARY E...,76.0
2856,369999,TO PROVIDE LOW COST THERAPY AND SERVICES TO CH...,13.0
2857,369999,TO PROVIDE ASSISTANCE IN THE DEVELOPMENT OF SO...,16.0


In [180]:
# rank the different descriptions and select the longest one
# note: if it has two descriptions that are the same length, it will arbitrarily select one of them
np_desc_populated["description_rank"] = np_desc_populated.groupby('nonprofit_id')['description_word_length'].rank()
max_desc_selected_df = np_desc_populated[np_desc_populated["description_rank"] == 1].copy(deep=True)
df_text = max_desc_selected_df[["nonprofit_id", "description"]].reset_index(drop=True)

In [181]:
# show if there are any duplicate records
df_text[df_text.duplicated(['nonprofit_id'])]

,nonprofit_id,description


In [182]:
df_text.head()


,nonprofit_id,description
0,336076,Donation to Boy Scout Troop 441. The Troop has...
1,5896,THE FOUNDATION'S PRIMARY OBJECTIVE IS TO SAVE ...
2,369999,TO PROVIDE LOW COST THERAPY AND SERVICES TO CH...
3,4002,PROVIDE JOB TRAINING JOB PLACEMENT AND SUITABL...
4,7597,50 CLIENTS SERVED. PURPOSE IS TO PROVIDE DAYCA...


In [183]:
# create a list of the description
data = df_text.description.values.tolist()
np.shape(data)

(5115,)

In [184]:
# tokenize and clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
np.shape(data_words)

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


(5115,)

In [185]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['donation', 'to', 'boy', 'scout', 'troop', 'the', 'troop', 'has', 'about', 'boys', 'and', 'adults']


In [186]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [187]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

data_words_nostops[0]

['donation', 'boy', 'scout', 'troop', 'troop', 'boys', 'adults']

In [188]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_words_bigrams[0]

['donation', 'boy', 'scout', 'troop', 'troop', 'boys', 'adults']

In [189]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

data_lemmatized[0]

['donation', 'boy', 'scout', 'troop', 'troop', 'boy', 'adult']

In [190]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2)]]


In [203]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [204]:
example_num = 36

In [205]:
df_text.loc[example_num,"description"]

'THE RESEARCH AWARD WILL BE GIVEN ANNUALLY TO RECOGNIZE FELLOWS IN SURGICAL CRITICAL CARE (SCC) AND ACUTE CARE SURGERY (ACS) FOR SIGNIFICANT RESEARCH ENDEAVORS AND CONTRIBUTIONS.'

In [206]:
data_lemmatized[example_num]

['research',
 'award',
 'give',
 'annually',
 'recognize',
 'fellow',
 'surgical',
 'critical',
 'care',
 'scc',
 'acute',
 'care',
 'surgery',
 'ac',
 'significant',
 'research',
 'endeavor',
 'contribution']

In [207]:
doc = lda_model.id2word.doc2bow(data_lemmatized[example_num])
doc_topics, word_topics, phi_values = lda_model.get_document_topics(doc, per_word_topics=True)
doc_topics
for val in doc_topics:
    print(val)
    print(lda_model.id2word[val[0]])

(0, 0.40047947)
adult
(1, 0.21448722)
boy
(2, 0.14349394)
donation
(3, 0.045525257)
scout
(4, 0.082420625)
troop
(5, 0.11359351)
abandon


In [208]:
def get_readable_topic(input_doc):
    if len(input_doc) > 1:
        doc = lda_model.id2word.doc2bow(input_doc)
        doc_topics = lda_model.get_document_topics(doc, per_word_topics=False)
        
        # initialize
        cur_val = 0
        cur_topic = 0

        # will loop over all of the doc topics
        # to find the one with the greatest 
        for val in doc_topics:
            if val[1] > cur_val:
                cur_val = val[1]
                cur_topic = val[0]
        lda_model.id2word[cur_topic]
        return lda_model.id2word[cur_topic]
    else:
        return None

index = 0
for doc in data_lemmatized:
    df_text.loc[index,"topic_result"] = get_readable_topic(doc)
    index = index + 1

In [209]:
df_text["topic_result"].unique()

array(['donation', 'scout', 'boy', 'abandon', 'troop', 'adult', None],
      dtype=object)

In [210]:
# export model results into unique file
df_for_csv = df_text[["nonprofit_id", "topic_result", "description"]]
df_for_csv.to_csv(f"{datetime.now().strftime('%Y_%m_%d_%H_%M')}_unique_count_{len(df_text['nonprofit_id'].unique())}_nonprofit_text_output.csv", sep="|", index=False)

In [211]:
len(df_text["nonprofit_id"].unique())

5115

In [212]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.602410328600436

Coherence Score:  0.3258117234514204


In [213]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/Library/Caches/pypoetry/virtualenvs/hackmt-2022--Ymf8L12-py3.10/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/aantonison/L

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.219093 -0.319102       1        1  28.334384
1      0.275198  0.278183       2        1  26.212194
0     -0.204794  0.027743       3        1  12.728093
4     -0.102557 -0.027882       4        1  12.345312
3     -0.092587  0.030214       5        1  12.195294
5     -0.094354  0.010844       6        1   8.184723, topic_info=              Term         Freq        Total Category  logprob  loglift
29         provide  2074.000000  2074.000000  Default  30.0000  30.0000
39           child   571.000000   571.000000  Default  29.0000  29.0000
41         service   810.000000   810.000000  Default  28.0000  28.0000
26         program   879.000000   879.000000  Default  27.0000  27.0000
55         purpose   508.000000   508.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
370           safe    46.597260    47.337421   Topic6  -4.7906   2.4871
215         museum    42.754329    43.497439   Topic6  -4.8767   2.4857
1892      athletic    41.748989    42.491758   Topic6  -4.9005   2.4853
203   recreational    39.094364    39.835428   Topic6  -4.9661   2.4841
141      operation    92.405511    94.757865   Topic6  -4.1059   2.4778

[225 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
5         5  0.982750     abandon
825       5  0.993393      access
6         3  0.998145    activity
143       3  0.987520  additional
0         5  0.860768       adult
...     ...       ...         ...
85        2  0.112800        work
645       4  0.990770       world
37        5  0.996428        year
1070      6  0.986059       young
180       6  0.998007       youth

[210 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 5, 4, 6])